In [ ]:
!pip install scikit-learn
!pip install matplotlib
!pip install pandas
!pip install gensim
!pip install nltk

In [ ]:
!pip install gensim
!pip install nltk

In [37]:
import pandas as pd

df_latin = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/latin.csv?raw=true')
df_engineer = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/engineer.csv?raw=true')

In [38]:
def answer_conc(df):

  df_questions = df[df['ParentId'].isna()]
  df_answers = df[df['ParentId'].notna()]

  df_answers = df_answers.drop(columns=['AcceptedAnswerId','ViewCount','Tags'])

  df_questions = df_questions.merge(df_answers[['Id','Text']], how='left', left_on='AcceptedAnswerId', right_on='Id', suffixes=('', '_Answer'))
  df_questions.rename(columns={'Text_Answer': 'Answer'}, inplace=True)
  df_questions = df_questions.drop(columns='Id_Answer')
  df_answers_reduced = df_answers.loc[df_answers.groupby('ParentId')['Score'].idxmax()]
  df_no_accepted_answer = df_questions[df_questions['AcceptedAnswerId'].isna()]

  for index, row in df_no_accepted_answer.iterrows():
    question_id = row['Id']
    highest_scored_answer = df_answers_reduced[df_answers_reduced['ParentId'] == question_id]
    if not highest_scored_answer.empty:
        df_questions.at[index, 'Answer'] = highest_scored_answer.iloc[0]['Text']
  return df_questions

In [39]:
def process_tags(tags):
    if pd.isna(tags):
      return []
    tags_list = tags.split('|')
    return tags_list[1:-1] if len(tags_list) > 2 else []

In [40]:
def big_clean(df):
  df = df.drop(columns=['LastActivityDate','CreationDate','PostTypeId','OwnerUserId','ContentLicense','CommentCount','AnswerCount'])
  df['Text'] = df['Title'].fillna('') + ' ' + df['Body'].fillna('').str.strip()
  df = df.drop(columns=['Body','Title'])

  questions = answer_conc(df)

  questions.rename(columns={'Text':'Question'},inplace=True)
  questions = questions.drop(columns=['ViewCount','AcceptedAnswerId','ParentId','Score'])

  questions['Tags'] = questions['Tags'].apply(process_tags)

  return questions

In [41]:
latin = big_clean(df_latin)
# ~1000 questions dropped
# latin = latin.dropna(subset=['Answer'])
latin

,Id,Tags,Question,Answer
0,1,"[classical-latin, meaning, vocabulary, conjunc...","Are ""-que"" and ""et"" equivalent? I was taught t...","The way I was taught was that, as a general r..."
1,2,"[contemporary-latin, pronouns, articles, trans...","Why is ""ille"" used in Winnie ille Pu and Hobbi...","It's true that in Classical Latin, ille is a ..."
2,4,"[agreement, predicate, gender, adjective, clas...",What gender should a predicate adjective be to...,Bennett's New Latin Grammar ( this link will ...
3,5,[pronunciation],What effect should a macron have on the sound ...,"In most modern texts, the whole purpose of us..."
4,7,"[etymology, particle, hebrew]","Does ""ad"" have its origin in Hebrew/Semitic la...","No, the similarity is almost certainly accide..."
...,...,...,...,...
6834,23420,[english-to-latin-translation],Four more loaves please This new question: How...,"In Nicholas Oulton's Book II, p.50: ""plures c..."
6835,23425,"[latin-to-english-translation, mathematics]","Mathematical Latin Help So, I'm a PhD student ...","I understand ""ut quotiens sit integer complex..."
6836,23429,"[latin-to-english-translation, translation-check]",Does “interranima” mean “inner soul”? I came a...,"No. Anima is the Latin word for soul, apart f..."
6837,23432,"[diphthong, paleography, ligatures]",What diphthongs are available are Unicode liga...,NaN


In [42]:
engineer = big_clean(df_engineer)
# ~2000 questions dropped
# engineer = engineer.dropna(subset=['Answer'])
engineer

,Id,Tags,Question,Answer
0,1,"[ipsec, ike, vpn]",What's the difference between IKE and ISAKMP? ...,"ISAKMP is part of IKE. (IKE has ISAKMP, SKEME..."
1,3,[qos],Quality of Service based on Layer 7 traffic Ho...,I think this depends heavily on the platform ...
2,4,"[cisco-catalyst, bgp]","Troubleshooting ""Down BGP"" connections Our net...",172259: May 6 14:43:06: %BGP-3-NOTIFICATION:...
3,5,"[ospf, security]",Securing OSPF with only passive interfaces I k...,One issue is that authentication ensures that...
4,6,"[juniper, policing, juniper-ex]",Aggregate Family Rate Limits on Juniper EX Try...,"You are looking for the ""Logical Interface (A..."
...,...,...,...,...
17516,85395,"[switch, network, networking]",Can I configure my Linux box as a virtual netw...,NaN
17517,85396,[tcp],"How does a server handle multiple requests, an...",NaN
17518,85398,"[switch, network, networking, layer3]",Core Switches Redundancy Problem I have networ...,"There are several ways to do this, depending ..."
17519,85401,"[juniper, ipv6, dhcp, dhcpv6, dhcp-snooping]",Juniper and EdgeCore dhcpv6-relay remote-id (o...,NaN


In [ ]:
#BOW unigram, tisztitást még megnézek rá hátha
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(engineer['Question'])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,00,000,0000,00000,000000,0000000,00000000,000000000,0000000000,00000000000,...,接口列表,活动路由,的路由,网关,网络掩码,网络目标,请求超时,跃点数,路由表,通过最多
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_average_cosine_similarity(model, documents):
    document_similarities = []

    for document in documents:
        words = word_tokenize(document.lower())
        vectors = [model.wv[word] for word in words if word in model.wv]

        if len(vectors) < 2:
            document_similarities.append(0.0)
            continue

        cosine_sim_matrix = cosine_similarity(vectors)
        upper_tri_indices = np.triu_indices_from(cosine_sim_matrix, k=1)
        similarities = cosine_sim_matrix[upper_tri_indices]

        average_similarity = np.mean(similarities)
        document_similarities.append(average_similarity)

    return document_similarities

def w2v_vectorize(df):
  corpus_text = ' '.join(latin['Question'])
  data = []
  for sentence in sent_tokenize(corpus_text):
      temp = []
      for word in word_tokenize(sentence):
        temp.append(word.lower())
      data.append(temp)

  cbow = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5, sg=0)
  skipgram = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5, sg=1)
  return (cbow,skipgram)

In [45]:
cbow, skipgram = w2v_vectorize(latin)

latin['Cosine Similarity (CBOW)'] = calculate_average_cosine_similarity(cbow, latin['Question'])
latin['Cosine Similarity (Skip-gram)'] = calculate_average_cosine_similarity(skipgram, latin['Question'])

cbow, skipgram = w2v_vectorize(engineer)

engineer['Cosine Similarity (CBOW)'] = calculate_average_cosine_similarity(cbow, engineer['Question'])
engineer['Cosine Similarity (Skip-gram)'] = calculate_average_cosine_similarity(skipgram, engineer['Question'])

In [46]:
engineer

,Id,Tags,Question,Answer,Cosine Similarity (CBOW),Cosine Similarity (Skip-gram)
0,1,"[ipsec, ike, vpn]",What's the difference between IKE and ISAKMP? ...,"ISAKMP is part of IKE. (IKE has ISAKMP, SKEME...",0.403661,0.582982
1,3,[qos],Quality of Service based on Layer 7 traffic Ho...,I think this depends heavily on the platform ...,0.419055,0.624165
2,4,"[cisco-catalyst, bgp]","Troubleshooting ""Down BGP"" connections Our net...",172259: May 6 14:43:06: %BGP-3-NOTIFICATION:...,0.423022,0.591208
3,5,"[ospf, security]",Securing OSPF with only passive interfaces I k...,One issue is that authentication ensures that...,0.424970,0.579540
4,6,"[juniper, policing, juniper-ex]",Aggregate Family Rate Limits on Juniper EX Try...,"You are looking for the ""Logical Interface (A...",0.430038,0.604160
...,...,...,...,...,...,...
17516,85395,"[switch, network, networking]",Can I configure my Linux box as a virtual netw...,NaN,0.421480,0.596350
17517,85396,[tcp],"How does a server handle multiple requests, an...",NaN,0.427845,0.619543
17518,85398,"[switch, network, networking, layer3]",Core Switches Redundancy Problem I have networ...,"There are several ways to do this, depending ...",0.348296,0.592543
17519,85401,"[juniper, ipv6, dhcp, dhcpv6, dhcp-snooping]",Juniper and EdgeCore dhcpv6-relay remote-id (o...,NaN,0.383547,0.539632


In [ ]:
# 2d ábrázolás - WIP

# cluster és tag korreláció